In [22]:
import json
import csv
from os import listdir
import time

In [12]:
def get_champ(game):
    game_data = []
    for p in game['participants']: 
        game_data.append(p['championId'])
    return game_data
def get_winner(game):
    if(game['teams'][0]['win'] == 'Win'):
        return 0
    else:
        return 1

In [50]:
def read_json_from_file(file):
    with open(file,encoding='utf8') as f:
        return json.load(f)
def write_list_of_list_to_csv(l,file):
    with open(file, "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(l)

In [62]:
def create_data(game_type):  
    #game_type = "aram","normal","soloduo","flex"
    
    files_list = listdir("data/"+game_type)
    
    try:
        with open("logs/"+game_type+".log","r") as f:
            last_game_used = f.read() 
    except FileNotFoundError:
        last_game_used = False
        
    if(last_game_used):
        last_game_used_index = files_list.index(last_game_used)
        files_list = files_list[last_game_used_index+1:len(files_list)]
    
    number_of_games = len(files_list)
    #(142 champs * 2 teams + 1 winner) * number_of_games
    
    data = []
    for n in range(number_of_games):
        temp = []
        for i in range(142*2+1):
            temp.append(0)
        data.append(temp)

    dict_champ_index = read_json_from_file("json_utils/champions_index.json")
    dict_champ_key = read_json_from_file("json_utils/champions_key.json")    

    for n in range(0,number_of_games):
        game = read_json_from_file("data/"+game_type+"/"+files_list[n])
        champions_list = get_champ(game)

        for c in range(5):
            champion_name = dict_champ_key[str(champions_list[c])]
            data[n][dict_champ_index[champion_name]] = 1                
        for c in range(5,10):
            champion_name = dict_champ_key[str(champions_list[c])]
            data[n][dict_champ_index[champion_name]+142] = 1       
            
        data[n][142*2] = get_winner(game)
        
    try:
        last_file = files_list[n]
        with open("logs/"+game_type+".log","w") as f:        
            f.write(last_file)
    except UnboundLocalError:
        print("no new games")
        
    return data

In [20]:
def invert(i):
    if(i==0):
        return [1]
    else:
        return [0]

def extend_dataset(dataset):
    res = []
    for d in dataset:
        res.append(d[142:284] + d[0:142] + invert(d[284]))
    return res

def create_header():
    dict_champ_index = read_json_from_file("json_utils/champions_index.json")
    header = []
    for c in range(0,142):
        header.append(dict_champ_index[str(c)]+"_team0")
    for c in range(0,142):
        header.append(dict_champ_index[str(c)]+"_team1")
    header.append("winner")
    return [header]

In [69]:
start = time.time()
data = create_data("aram")
print(time.time() - start)

#data += extend_dataset(data)
#data = create_header() + data
if(len(data) != 0):
    write_list_of_list_to_csv(data,"csv/data_aram.csv")

no new games
0.0813148021697998


In [70]:
len(data)

0

In [7]:
data_normal = create_data("normal")
data_soloduo = create_data("soloduo")

write_list_of_list_to_csv(data_normal+data_soloduo,"csv/data_classic_soloduo.csv")